In [2]:
import os
import glob
import csv
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# From ultralytics import YOLO (for YOLO 11)
from ultralytics import YOLO

# TorchVision v2 transforms
from torchvision.transforms import v2
from torchvision.transforms.v2 import functional as F

In [ ]:
# Renaming Images to match the format FFFFF_IIIII.png

def rename_images(base_dir, start_folder_index=43):
    """
    Iterates through each folder under 'base_dir' and renames the contained images 
    to match the format FFFFF_IIIII.png, where:
      - FFFFF is a five-digit folder index (starting at 'start_folder_index')
      - IIIII is a five-digit image index (0-based) per folder
    """
    folder_paths = sorted(
        [f for f in glob.glob(os.path.join(base_dir, "*")) if os.path.isdir(f)]
    )

    current_folder_index = start_folder_index
    for folder_path in folder_paths:
        folder_name_formatted = f"{current_folder_index:05d}"
        
        image_paths = sorted(
            [img for img in glob.glob(os.path.join(folder_path, "*.png"))]
        )
        
        image_counter = 0
        for image_path in image_paths:
            image_name_formatted = f"{image_counter:05d}"
            new_name = f"{folder_name_formatted}_{image_name_formatted}.png"
            new_path = os.path.join(folder_path, new_name)
            os.rename(image_path, new_path)
            image_counter += 1
        
        current_folder_index += 1

In [ ]:
# Data Augmentation for Traffic Signs

class TrafficSignAugmentationPipeline:
    """
    Simple augmentation pipeline for traffic signs using torchvision.transforms.v2.
    Skips flips and large rotations since orientation is crucial for traffic signs.
    Emphasizes brightness, contrast, and minimal geometric changes.
    """
    def __init__(self):
        self.augment = v2.Compose([
            # Convert to Tensor first, so subsequent operations work on tensor data
            v2.ToImage(),
            
            # Random resizing within a range
            v2.RandomResize(
                min_size=int(0.5 * 256),
                max_size=int(1.0 * 256),
                interpolation=Image.BICUBIC
            ),
            
            # Adjust brightness/contrast to simulate varying lighting conditions
            v2.ColorJitter(
                brightness=0.4,
                contrast=0.4,
            ),
        
            
            # Convert to float and scale from [0, 255] to [0, 1]
            v2.ToDtype(torch.float32, scale=True),
        ])
        
    def __call__(self, img):
        """
        Applies all the transforms to the input image (tensor or PIL).
        Returns an augmented tensor image.
        """
        return self.augment(img)


def augment_images(base_dir, augmentation_pipeline):
    """
    Locates all .png images recursively in 'base_dir' and applies the provided 
    augmentation pipeline to each image. Saves augmented images in place by default.
    """
    image_paths = sorted(glob.glob(os.path.join(base_dir, "**", "*.png"), recursive=True))
    
    for img_path in image_paths:
        img = Image.open(img_path).convert("RGB")
        
        # Convert to tensor with v2.ToImage internally, or simply do the pipeline call
        img_tensor = F.to_tensor(img).float()
        aug_img = augmentation_pipeline(img_tensor)
        
        # Convert the augmented tensor back to PIL and overwrite the original
        aug_pil = F.to_pil_image(aug_img)
        aug_pil.save(img_path)

        # Alternatively, to save a separate augmented copy, uncomment:
        # new_path = img_path.replace(".png", "_aug.png")
        # aug_pil.save(new_path)

In [1]:
# YOLO 11 Detection and Visualization

def detect_and_visualize(base_dir, model_path="yolo11n.pt", conf_threshold=0.25, output_dir="output"):
    """
    Loads a YOLO 11 model (e.g., yolo11n.pt) via ultralytics and applies it 
    to each .png image under 'base_dir'.
    
    Returns detection results as (image_path, width, height, [boxes]) in a YOLO-like format:
      boxes -> [(class_id, x_center, y_center, box_width, box_height), ...]
    Visualizes and saves every 50th detection with bounding boxes in the output directory.
    """

    # Load YOLO 11 model from ultralytics
    model = YOLO(model_path)

    # The detection results are collected here
    detection_results = []

    all_image_paths = sorted(glob.glob(os.path.join(base_dir, "**", "*.png"), recursive=True))
    for idx, img_path in enumerate(all_image_paths):
        img = Image.open(img_path).convert("RGB")
        width, height = img.size

        # Perform inference using the model
        results = model.predict(source=img, conf=conf_threshold)

        # YOLO results come in a list, one per image. There's only one image here.
        boxes_info = []
        if len(results) > 0:
            r = results[0]
            # 'r.boxes' is a Boxes object containing bounding box coords, class, conf
            for box in r.boxes:
                # box.xyxy -> tensor of shape [1,4]
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                w_box = x2 - x1
                h_box = y2 - y1
                x_center = x1 + w_box / 2
                y_center = y1 + h_box / 2
                cls_id = int(box.cls.item())
                boxes_info.append((cls_id, x_center, y_center, w_box, h_box))

        detection_results.append((img_path, width, height, boxes_info))

        # Visualize and save every 50th image
        if idx % 50 == 0:
            fig, ax = plt.subplots(1)
            ax.imshow(img)
            for (cls_id, x_c, y_c, bw, bh) in boxes_info:
                rect = plt.Rectangle(
                    (x_c - bw/2, y_c - bh/2),
                    bw, bh,
                    fill=False,
                    color="red",
                    linewidth=2
                )
                ax.add_patch(rect)
            plt.title(f"Detections for {os.path.basename(img_path)}")

            # Save the figure
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            output_path = os.path.join(output_dir, f"detection_{idx}.png")
            plt.savefig(output_path)
            plt.close(fig)
        
    return detection_results

In [ ]:
# Converting YOLO-Format Boxes to dataset's CSV

def convert_detections_to_csv(detection_results, csv_filename="annotations.csv"):
    """
    Writes a CSV with columns:
      Width, Height, Roi.X1, Roi.Y1, Roi.X2, Roi.Y2, ClassId, Path
    Converting YOLO-like (x_center, y_center, w_box, h_box) to absolute 
    coordinates (x1, y1, x2, y2).
    """
    header = ["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"]
    
    with open(csv_filename, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        
        for (img_path, img_w, img_h, boxes) in detection_results:
            for (class_id, x_center, y_center, bw, bh) in boxes:
                x1 = x_center - bw / 2
                y1 = y_center - bh / 2
                x2 = x_center + bw / 2
                y2 = y_center + bh / 2
                
                row = [
                    img_w,
                    img_h,
                    int(x1),
                    int(y1),
                    int(x2),
                    int(y2),
                    class_id,
                    img_path
                ]
                writer.writerow(row)

In [ ]:
########################
# 5) Main Execution Flow
########################
if __name__ == "__main__":
    base_dir = r"C:\Users\polat\Desktop\DATA" # Path to the dataset
    model_path = "traffic_signs.pt"   # YOLO 11 weight file
    
    # Step A: Rename images in existing folders (optional)
    # rename_images(base_dir, start_folder_index=43)
    
    # Step B: Augment images
    # augmentation_pipeline = TrafficSignAugmentationPipeline()
    # augment_images(base_dir, augmentation_pipeline)
    
    # Step C: Detection with YOLO 11
    detections = detect_and_visualize(base_dir, model_path=model_path, conf_threshold=0.25) # Saved in output folder
    
    # Step D: Conversion to CSV
    # convert_detections_to_csv(detections, csv_filename="annotations.csv")



0: 640x640 1 sign, 12.0ms
Speed: 3.1ms preprocess, 12.0ms inference, 62.6ms postprocess per image at shape (1, 3, 640, 640)

0: 608x640 1 sign, 66.8ms
Speed: 2.0ms preprocess, 66.8ms inference, 2.1ms postprocess per image at shape (1, 3, 608, 640)

0: 640x640 1 sign, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 544x640 1 sign, 56.4ms
Speed: 2.1ms preprocess, 56.4ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)

0: 576x640 1 sign, 58.5ms
Speed: 2.0ms preprocess, 58.5ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 sign, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 608x640 1 sign, 16.1ms
Speed: 3.0ms preprocess, 16.1ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 sign, 11.1ms
Speed: 3.0ms preprocess, 11.1ms inference, 2.0ms postprocess per image at shape (1, 3, 608, 640)

0: 608

In [11]:
# After replacing \ with /, and path with Test.csv alike path
# the following code is for extracting the class id from the path column
import pandas as pd
df = pd.read_csv("annotations.csv")
df["Path"]

# take the value between first and second / characters
df["Path"].str.split("/", expand=True)[1]

# put this values into the ClassId column
df["ClassId"] = df["Path"].str.split("/", expand=True)[1]

# save the updated dataframe to a new csv file
df.to_csv("annotations_updated.csv", index=False)

# Manual Annotation using CVAT

In [16]:
import os
import csv
import cv2
import xml.etree.ElementTree as ET
import numpy as np
from albumentations import Compose, RandomBrightnessContrast, Rotate, Blur, Perspective

# Paths
original_dir = r"C:\Users\polat\Desktop\62-69"
pascalvoc_dir = r"C:\Users\polat\Desktop\pascalvoc"
augmented_dir = r"C:\Users\polat\Desktop\62-69_Augmented"
output_csv_path = r"C:\Users\polat\Desktop\pascalvoc\instances_augmented.csv"

# Albumentations transform pipeline
transform_pipeline = Compose(
    [
        RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
        Rotate(limit=15, border_mode=cv2.BORDER_REFLECT, p=0.5),
        Blur(blur_limit=3, p=0.2),
        Perspective(scale=(0.03, 0.06), p=0.3),
    ],
    bbox_params={
        "format": "pascal_voc",  # (xmin, ymin, xmax, ymax)
        "label_fields": ["labels"],
    },
)

def parse_pascal_voc(xml_path):
    """
    Parses a Pascal VOC XML file to extract filename, image size,
    and bounding box data in pascal_voc format.
    Returns:
      filename (str): Image file name (e.g., "62-1.png")
      width (int), height (int): Dimensions from the <size> tag
      bboxes (list of lists): Each box in [xmin, ymin, xmax, ymax]
      labels (list of str): Class labels corresponding to each box
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.findtext("filename")
    width = int(root.find("./size/width").text)
    height = int(root.find("./size/height").text)

    bboxes = []
    labels = []
    for obj in root.findall("object"):
        class_name = obj.findtext("name")  # e.g., "62"
        bndbox = obj.find("bndbox")
        xmin = float(bndbox.findtext("xmin"))
        ymin = float(bndbox.findtext("ymin"))
        xmax = float(bndbox.findtext("xmax"))
        ymax = float(bndbox.findtext("ymax"))

        # Albumentations expects [xmin, ymin, xmax, ymax] for pascal_voc
        bboxes.append([xmin, ymin, xmax, ymax])
        labels.append(class_name)

    return filename, width, height, bboxes, labels

# Opening a CSV in write mode (overwriting or creating fresh)
# If needing to append to an existing dataset, consider using 'a' instead of 'w'
with open(output_csv_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Writing header row
    csv_writer.writerow(["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"])

    # Iterating over each class folder in original_dir (e.g., "62", "63", etc.)
    for class_folder in os.listdir(original_dir):
        class_folder_path = os.path.join(original_dir, class_folder)
        if not os.path.isdir(class_folder_path):
            continue

        # Zero-pad the class name (e.g., "00062" for "62")
        padded_class_name = str(class_folder).zfill(5)

        # Creating a parallel folder in augmented_dir
        augmented_class_folder = os.path.join(augmented_dir, class_folder)
        os.makedirs(augmented_class_folder, exist_ok=True)

        # Listing image files in the current class folder
        image_files = [
            f for f in os.listdir(class_folder_path)
            if os.path.isfile(os.path.join(class_folder_path, f))
               and f.endswith((".png"))
        ]

        # Processing each image in this class folder
        for image_index, image_file in enumerate(image_files):
            # Matching an XML file in pascalvoc_dir with the same base name
            image_base, _ = os.path.splitext(image_file)
            xml_filename = image_base + ".xml"
            xml_path = os.path.join(pascalvoc_dir, xml_filename)
            if not os.path.exists(xml_path):
                print(f"Warning: No XML annotation found for {image_file}. Skipping bounding box parsing.")
                continue

            # Parsing the Pascal VOC XML
            try:
                filename, orig_width, orig_height, bboxes, labels = parse_pascal_voc(xml_path)
            except Exception as e:
                print(f"Error parsing XML for {xml_path}: {e}")
                continue

            # Loading the image
            image_path = os.path.join(class_folder_path, filename)
            original_image = cv2.imread(image_path)
            if original_image is None or not isinstance(original_image, np.ndarray):
                print(f"Warning: Unable to read image {image_path}. Skipping.")
                continue

            # Zero-padded index for this image
            padded_image_index = str(image_index).zfill(5)

            # Creating multiple augmentations (e.g., 20)
            for aug_index in range(20):
                try:
                    transformed = transform_pipeline(
                        image=original_image,
                        bboxes=bboxes,
                        labels=labels
                    )
                except Exception as e:
                    print(f"Augmentation error for {image_path}: {e}")
                    continue

                augmented_image = transformed["image"]
                augmented_bboxes = transformed["bboxes"]
                augmented_labels = transformed["labels"]

                # Zero-padded augmentation index
                padded_aug_index = str(aug_index).zfill(5)

                # Constructing an output filename:
                # <padded_class_name>_<padded_image_index>_<padded_aug_index>.png
                # e.g., 00062_00000_00000.png
                aug_filename = f"{padded_class_name}_{padded_image_index}_{padded_aug_index}.png"
                save_path = os.path.join(augmented_class_folder, aug_filename)

                # Saving the augmented image
                cv2.imwrite(save_path, augmented_image)

                # Recording bounding boxes in the CSV
                height, width = augmented_image.shape[:2]
                for abox, label in zip(augmented_bboxes, augmented_labels):
                    # abox is [xmin, ymin, xmax, ymax] in pascal_voc format
                    x1 = int(abox[0])
                    y1 = int(abox[1])
                    x2 = int(abox[2])
                    y2 = int(abox[3])

                    # Constructing a relative path for CSV
                    # e.g., Train/62/00062_00000_00000.png
                    relative_path = f"Train/{class_folder}/{aug_filename}"

                    # Writing row: Width, Height, Roi.X1, Roi.Y1, Roi.X2, Roi.Y2, ClassId, Path
                    # "ClassId" can be the numeric label if needed, or the textual label
                    csv_writer.writerow([
                        width,
                        height,
                        x1,
                        y1,
                        x2,
                        y2,
                        label,  # or int(label) if classes are purely numeric
                        relative_path
                    ])

print("Augmentation process completed with Pascal VOC annotations.")

Augmentation process completed with Pascal VOC annotations.


# Test dataset

In [ ]:
from albumentations import (
    RandomGamma, GaussNoise, ShiftScaleRotate, GaussianBlur
)

# Paths
original_dir = r"C:\Users\polat\Desktop\62-69"
pascalvoc_dir = r"C:\Users\polat\Desktop\pascalvoc"
test_images_dir = r"C:\Users\polat\Desktop\Test_Augmented\Test"
output_csv_path = r"C:\Users\polat\Desktop\pascalvoc\test_instances_augmented.csv"

# Number of images to pick randomly from each class folder
images_per_class = 5

# Index from which test filenames begin
test_index_start = 13424

# New augmentation pipeline
new_transform_pipeline = Compose(
    [
        RandomGamma(gamma_limit=(80, 120), p=0.6),
        GaussNoise(p=0.3),
        ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.2,
            rotate_limit=10,
            border_mode=cv2.BORDER_REFLECT,
            p=0.5
        ),
        GaussianBlur(blur_limit=(3, 7), p=0.3)
    ],
    bbox_params={
        "format": "pascal_voc",  # bounding boxes in [xmin, ymin, xmax, ymax]
        "label_fields": ["labels"]
    }
)

def parse_pascal_voc(xml_path):
    """
    Parses a Pascal VOC XML file to extract:
      - filename (str)
      - image size (width, height)
      - bounding box data in pascal_voc format ([xmin, ymin, xmax, ymax])
      - labels (class names)
    """
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.findtext("filename")
    width = int(root.find("./size/width").text)
    height = int(root.find("./size/height").text)

    bboxes = []
    labels = []
    for obj in root.findall("object"):
        class_name = obj.findtext("name")
        bndbox = obj.find("bndbox")
        xmin = float(bndbox.findtext("xmin"))
        ymin = float(bndbox.findtext("ymin"))
        xmax = float(bndbox.findtext("xmax"))
        ymax = float(bndbox.findtext("ymax"))

        bboxes.append([xmin, ymin, xmax, ymax])
        labels.append(class_name)

    return filename, width, height, bboxes, labels

# Ensuring test output folder exists
os.makedirs(test_images_dir, exist_ok=True)

# Opening a new CSV file (write mode)
with open(output_csv_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    # Writing the CSV header
    csv_writer.writerow(["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"])
    
    # Counter for naming test images starting from test_index_start
    test_index = test_index_start

    # Iterating over each class folder in original_dir
    for class_folder in os.listdir(original_dir):
        class_folder_path = os.path.join(original_dir, class_folder)
        if not os.path.isdir(class_folder_path):
            continue

        # Gathering all .png images in this class folder
        image_files = [
            f for f in os.listdir(class_folder_path)
            if os.path.isfile(os.path.join(class_folder_path, f)) and f.lower().endswith(".png")
        ]

        # Randomly picking a subset of images
        if len(image_files) > images_per_class:
            selected_images = random.sample(image_files, images_per_class)
        else:
            selected_images = image_files  # If fewer than images_per_class, just use them all

        # Processing each selected image for the new augmentation pipeline
        for image_index, image_file in enumerate(selected_images):
            # Match to the corresponding XML annotation
            image_base, _ = os.path.splitext(image_file)
            xml_filename = image_base + ".xml"
            xml_path = os.path.join(pascalvoc_dir, xml_filename)
            if not os.path.exists(xml_path):
                print(f"Warning: No XML annotation found for {image_file}. Skipping.")
                continue

            try:
                filename, orig_w, orig_h, bboxes, labels = parse_pascal_voc(xml_path)
            except Exception as e:
                print(f"Error parsing XML for {xml_path}: {e}")
                continue

            # Loading the image from the original folder
            image_path = os.path.join(class_folder_path, filename)
            original_image = cv2.imread(image_path)
            if original_image is None or not isinstance(original_image, np.ndarray):
                print(f"Warning: Unable to read image {image_path}. Skipping.")
                continue

            # Generating multiple new augmentations
            for aug_index in range(10):  # Example: create 10 new variants
                try:
                    transformed = new_transform_pipeline(
                        image=original_image,
                        bboxes=bboxes,
                        labels=labels
                    )
                except Exception as e:
                    print(f"Augmentation error for {image_path}: {e}")
                    continue

                aug_image = transformed["image"]
                aug_bboxes = transformed["bboxes"]
                aug_labels = transformed["labels"]

                # Constructing a new filename for the test image
                new_filename = f"{test_index}.png"
                save_path = os.path.join(test_images_dir, new_filename)

                # Saving the new augmented image
                cv2.imwrite(save_path, aug_image)

                # Recording bounding boxes for each augmented object
                new_height, new_width = aug_image.shape[:2]
                for abox, lbl in zip(aug_bboxes, aug_labels):
                    x1 = int(abox[0])
                    y1 = int(abox[1])
                    x2 = int(abox[2])
                    y2 = int(abox[3])

                    # Path in the CSV is "Test/<number>.png"
                    relative_path = f"Test/{new_filename}"

                    # Storing the row
                    csv_writer.writerow([
                        new_width,
                        new_height,
                        x1,
                        y1,
                        x2,
                        y2,
                        lbl,
                        relative_path
                    ])

                # Incrementing the test image index for the next file
                test_index += 1

print("Partial augmentation with new filenames (starting at 13424) completed.")

Partial augmentation with new filenames (starting at 13424) completed.


# Training the model further

In [1]:
from ultralytics import YOLO

# Loading the previously trained model
model = YOLO("models/traffic.pt")

# Resuming training using the same configuration file
model.train(
    data="yolo_model_config.yaml",  # Same dataset YAML
    epochs=20,                      # Additional epochs
    device=0,
    name="traffic_continued"
)

FileNotFoundError: [Errno 2] No such file or directory: 'traffic.pt'